# Comparing degenerations of Schubert varieties

*Authors: George Balla, Daniel Corey, Igor Makhlin, and Victoria Schleis*

In this notebook accompanying our paper "Regular subdivisions, bounds on initial ideals, and categorical limits", we compute degenerations of Schubert varieties $X(B)$.  We compare the initial degenerations $\mathrm{in}_w(I)$ and the lower bound $I^w$ (as discussed in Section 3.2) of the ideal $I = I(X_B)$ of a Schubert variety, for all Schubert varieties contained in the Grassmannians $\operatorname{Gr}(2,7)$, $\operatorname{Gr}(3,6)$ and $\operatorname{Gr}(3,7)$.    

In [ ]:
using Oscar;
using Combinatorics;
pm = Polymake;

In [ ]:
include("InitialIdealsRegularSubdivisions.jl")
include("gfan_functions.jl")


## Schubert varieties

Let $1\leq r \leq n$ be integers and let $B = (b_1, \ldots, b_r)$ be a $r$-element list of distinct elements of $\{1,\ldots,n\}$ in increasing order. The Schubert variety $X_B$ is the subvariety of $\operatorname{Gr}(r,n)$ cut out by the ideal $I_{r,n} + \langle p_{A} :  A \nleq B \rangle$. Equivalently, $X_B$ is cut out in $\mathbb P(\mathbb C^{\{A : A\leq B\}})$ by
$$
    I(X_B) = (I_{k,n} + \langle x_{A} : A\nleq B \rangle ) \cap \mathbb C[x_A : A\leq B].
$$
In the function `schubert_variety` below, we construct the defining ideal of $X_B$. There is an option `elim` which, if marked `true`, will eliminate the variables $x_{A}$ for  $A \nleq B$ and output $I(X_B)$. Note that this coincides with the image of in $I_{r,n}$ under setting all such $x_{A}$ to $0$. 


---

**Function:** `partial_order_lists(A, B)`

*Description*: Given two vectors `A` and `B` of integers, returns `true` if both vectors have the same length and, coordinate-wise, all entries of `A` are smaller than all entries of `B`

---

**Function:** `lex_index(r, n)`

*Description*: For two integers `r` $<$ `n`, the function computes a dictionary assigning to each subset of size `r` of the integers $[n]$ its index in the lexicographical ordering of the subsets. 

---

**Function:** `remove_vars(I, vars_to_keep_index)`

*Description*: For a polynomial ideal `I` over the polynomial ring $K[x_1, \dots, x_n]$, returns the projection of `I` to the polynomial ring $K[x_1, \dots, x_m]$ where the variables in $K[x_1, \dots, x_m]$ are indexed by the vector `vars_to_keep_index`

---

**Function:** `schubert_variety(r, n, B, elim=false)`

*Description*: For two integers `r` $<$ `n` and a vector of integers `B` of length `r` with distinct entries up to `n`, computes the ideal of the Schubert variety $X_B$ as a subvariety of the Grassmannian $\operatorname{Gr}(r,n)$. This means that the ideal is given in Plücker coordinates. There is an option `elim` which, if marked `true`, will eliminate the variables $p_{A}$ for  $A \nleq B$.

In [ ]:
function partial_order_lists(A, B)
    if length(A) != length(B)
        return false
    elseif all([A[i] <= B[i] for i in 1:length(A)])
        return true
    else
        return false
    end
end

function lex_index(r, n)
    A = sort!(subsets(n,r))
    return Dict(A[i] => i for i in 1:length(A))
end

function remove_vars(I, vars_to_keep_index)
    R = base_ring(I)
    x = gens(R)
    m = length(vars_to_keep_index)
    vars_to_keep = x[vars_to_keep_index]
    S, y = polynomial_ring(coefficient_ring(R), ["y$(i)" for i in 1:m])
    phi = hom(S, R, vars_to_keep)
    return preimage(phi, I)
end

function schubert_variety(r, n, B, elim=false)
    I = [A for A in subsets(n, r) if !partial_order_lists(A, B)]
    sort!(setdiff( subsets(n, r), I))
    ncr = binomial(n,r) 
    R, x = polynomial_ring(QQ, ncr)
    Gr = grassmann_pluecker_ideal(R, r, n)
    S = base_ring(Gr)
    R = forget_grading(S)
    phi = hom(R, S, gens(S))  
    Gr = preimage(phi, Gr)
    x = gens(R)
    ind = lex_index(r, n)
    if length(I) != 0
        J = [x[ind[A]] for A in I]
    else
        J =[]
    end
    Jc = sort!([ind[A] for A in keys(ind) if !(A in I) ])
    K = Gr + ideal(R, J)
    if elim
        if length(J) != 0
            Kelim = eliminate(K, J)
        else
            Kelim = K
        end
        return remove_vars(Kelim, Jc)
    else
        return K
    end
end

*Example*: We compute ideal of the Schubert variety $X_{2,4,7} \subseteq Gr(3,7)$.

In [ ]:
r=3; n=7; B=[2,4,7];
#r=3; n=7; B=[3,6,7];
IS = schubert_variety(r, n, B, true)

## Point configurations for Schubert varieties

In this section we compute the point configurations of Schubert varieties and Grassmannians. 

The point configuration of the Grassmannian $\operatorname{Gr}(r,n)$ consists of the vertices of the hypersimplex $\Delta(r,n)$, and these vertices are labeled by the $r$ element subsets of $\{1,\ldots,n\}$. The point configuiration of the Schubert variety $X_B$ consists of vertices corresponding to the subsets $A$ satisfying $A\leq B$.


---

**Function:** `grassmannian_lineality_space(r, n)`

*Description*: Computes the point configuration of the Grassmannian of rank `r` and size `n`. 

---

**Function:** `schubert_lineality_space(r, n, B)`

*Description*: Computes the point configuration of the Schubert variety $X_B$ contained in the Grassmannian $\operatorname{Gr}$.

In [ ]:
function grassmannian_lineality_space(r, n)
    L = zeros(Int, n, binomial(n,r))
    j=1
    for A in sort!(subsets(n, r))
        L[A, [j]] = ones(Int, r, 1)
        j+=1
    end
    return L
end

function schubert_lineality_space(r, n, B)
    I = [A for A in sort!(subsets(n, r)) if partial_order_lists(A, B)]
    L = zeros(Int, length(I), n)
    j=1
    for A in I
        L[[j], A] = ones(Int, 1, r)
        j+=1
    end
    return L
end

*Example*: We compute the point configuration of the Schubert variety $X_{2,4,7} \subseteq Gr(3,7)$.

In [ ]:
point_conf = schubert_lineality_space(r, n, B)

## The lower bound $I_w$ and the fan $\Omega(I)$

We briefly recall the definitions of $I_w$ and $\Omega(I)$. Let $\Delta = \mathcal{A}(I)$ be the point configuration of $I$ from the paper, which can be computed for Grassmannians and Schubert varieties using the functions in the preceeding section. A vector $w\in \mathbb{R}^{n}$ induces a regular subdivision $\mathrm{subd}_{w}(\Delta)$ of $\Delta$. We can form the ideals 

$$\{\tilde{I}_{B} \, : \, B \text{ is a maximal cell of } \mathrm{subd}_{w}(\Delta)\}$$

and the ideal they generate in $S$ is

$$I_{w} = \sum_{B \text{ maximal}} \tilde{I}_{B}. $$

---

For an ideal $I$ with point configuration $\Delta$, we define $\Omega(I)$ by

$$ \Omega(I) = \{w \in \mathbb{R}^{n} \, : \, \mathrm{in}_{w} I = I_{w} \}. $$

This is a subfan of the secondary fan $\mathrm{Sec}(\Delta)$.  We also consider the fan

$$ \Omega_{t}(I) = \{w \in \mathrm{Trop}(I) \, : \, \mathrm{in}_{w} I = I_{w} \}. $$



---

**Function:** `Omega_fan_t(I, Delta, Sec)`

*Description*: Given an ideal `I` of a polynomial ring, `Delta` the point configuration of `I`, and `Sec` is either the secondary fan of `Delta` or the topicalization of `I`, returns the fan $\Omega_t(I)$ as a pair of rays and cones. 

Here, `Sec` can be given as a `PolyhedralFan` or as a pair `[rays_Delta, cones_Delta]` where `rays_Delta` is a `Vector{RayVector}` containing the rays and `cones_Delta` is a `IncidenceMatrix` recording the cones of `Sec`. The latter is useful when `Sec` is computed up to symmetry. 

Additional *examples* for this function can be found in `Computing_I_w_and_Omega.ipynb`.

In [ ]:
function ray_cone_to_reps(rs, cs)
    n_cones, n_rays = size(cs)
    d = length(rs[1])
    reps = []
    for i in 1:n_cones
        w = [rs[j] for j in 1:n_rays if cs[i,j]]
        if length(w) == 0
            w = zeros(Int, d)
        else
            w = sum(w)
        end
        push!(reps, w)
    end
    return reps
end

function Omega_fan_t(I, Delta, Sec)
    if typeof(Sec) == PolyhedralFan{QQFieldElem}
        rays_Delta = rays_modulo_lineality(Sec)[1];
        cones_Delta = cones(Sec);
    else
        rays_Delta, cones_Delta = Sec
    end

    n_cones_Delta, n_rays_Delta = size(cones_Delta)
    reps = ray_cone_to_reps(rays_Delta, cones_Delta)

    tests = [true]
    for w in reps[2:length(reps)]
        w = -Vector{Int64}(pm.common.primitive(w))
        init_w_I = initial(I, nu_t, w)
        I_w = ideal_w(I,  w)
        #I_w = ideal_w(I,  w, Delta)
        push!(tests, init_w_I == I_w)
        #@show tests
    end
    inside_Omega = [i for i in 1:length(reps) if tests[i]];
    return [rays_Delta, cones_Delta[inside_Omega, :]]  
end

*Example*: We compute the fan  $\Omega_{t}(X_{2,4,7})$ using our computations of the ideal of the Schubert variety and its lineality space above, and the pre-computed tropical variety to source weight vectors. Further, we demonstrate our file management for the remainder of the notebook.

The tropicalization of $X_B$ is pre-computed as `schubert_r3_n7_B247.trop` and loaded from the file path `schubert/tropicalizations_nocones` provided in the repository. The resulting computed fan is then saved as `Omega_tropical_no_saturation_cones_r3_n7_B247.mrdi` in the folder `schubert/omega_in_tropical_fan`.

In [ ]:
Bs = join([string(a) for a in B])
just_file = "schubert_r$(r)_n$(n)_B$(Bs).trop"
file_name = joinpath("precomputed_data/schubert/", "tropicalizations_nocones", just_file)
rs, cs = parse_tropical_file(file_name, true)
ray_cone_to_reps(rs, cs)
O = Omega_fan_t(IS, point_conf, [rs, cs])

To save the cone data, run 
```
just_file = "Omega_tropical_no_saturation_cones_r$(r)_n$(n)_B$(Bs).mrdi"
file_name = joinpath("precomputed_data/schubert/", "omega_in_tropical_fan", just_file)
save(file_name, O[2]);
```
In the next cell, we show that the saved data agrees with the one just computed. 

In [ ]:
just_file = "Omega_tropical_no_saturation_cones_r$(r)_n$(n)_B$(Bs).mrdi"
file_name = joinpath("precomputed_data/schubert/", "omega_in_tropical_fan", just_file)
O_saved = load(file_name)
O[2] == O_saved

## Comparing $\mathrm{in}_w I$ and $I_w$

The main goal of this notebook is to analyze computationally for which Schubert varieties $X_B$ and $w \in \mathrm{Trop}(X_B)$ we have $\mathrm{in}_w I = I_w$ where $I = I(X_{B})$. In particular, we compare $\mathrm{Trop}(X_B)$ and $\Omega(X_B)$. 

From [Cor] we know that for each Schubert variety $X_{B}$ with ideal $I = I(X_{B})$ in $\operatorname{Gr}(2,7)$, $\operatorname{Gr}(3,6)$ and $\operatorname{Gr}(3,7)$, we have that the saturations of $\mathrm{in}_{w}I^{\mathrm{sat}} = I_w^{\mathrm{sat}}$, i.e., that the very affine tropicalization and the very affine fan $\Omega(X_B)$ coincide. 

We now investigate the unsaturated case. The fans $\Omega(X_B)$ are computed as in the example above. While the computation of each fan only takes about two minutes, there are, in general, many Schubert varieties $X_B$. Thus, for the sake of running time when running this notebook, we only include the case $r =2$, $n =7$. 

For all other ranks and dimensions, computations proceed exactly analogous and can thus be easily amended. We further provide all intermediate steps and computational results in the folder `schubert` in this github repository, here. 

We start by computing the ideals of all Schubert varieties in $\operatorname{Gr}(2,7)$ using `schubert_variety`, and save all nontrivial ideals in a format compatible for reading into `gfan` using the function `ideal_to_gfan`. The resulting files are saved in the folder `schubert/initial_files`. This is achieved by the following block of code.  
```
r = 2; n =7;
for B in sort!(subsets(n,r))
    if last(B) != n
        continue
    end
    Bs = join([string(a) for a in B])
    S = schubert_variety(r, n, B, true)
    if iszero(S) 
        continue
    end
    just_file = "schubert_r$(r)_n$(n)_B$(Bs).dat"
    file_name = joinpath("precomputed_data/schubert", "initial_files", just_file)
    ideal_to_gfan(S, true, file_name)
end
```
In the following code block, we verify that the precomputed files agree with those generated by this code.

In [ ]:
r = 2; n =7;
tests = []
for B in sort!(subsets(n,r))
    if last(B) != n
        continue
    end
    Bs = join([string(a) for a in B])
    S = schubert_variety(r, n, B, true)
    if iszero(S) 
        continue
    end
    just_file = "schubert_r$(r)_n$(n)_B$(Bs).dat"
    file_name = joinpath("precomputed_data/schubert", "initial_files", just_file)
    generated_file = ideal_to_gfan(S, false)
    pre_computed = read(file_name, String)
    push!(tests, pre_computed == generated_file)
end
@show all(tests);


To compute the tropical varieties of all of the Schubert varieties we just computed, we want to use the `gfan` algorithm `tropicaltraverse`. This requires an intermediate pre-processing step: We compute starting Groebner cones for all ideals arising from Schubert varieties using the function `gfan_tropicalstartingcone`. These are saved in the folder `schubert/start_cones`. Here is the code that creates these files. 
```
r = 2; n = 7;
for B in sort!(subsets(n,r))
    if last(B) != n
        continue
    end
    Bs = join([string(a) for a in B])
    S = schubert_variety(r, n, B, true)
    if iszero(S)
        continue
    end
    in_just_file = "schubert_r$(r)_n$(n)_B$(Bs).dat"
    in_file_name = joinpath("precomputed_data/schubert", "initial_files", in_just_file)
    out_just_file = "schubert_r$(r)_n$(n)_B$(Bs).dat"
    out_file_name = joinpath("precomputed_data/schubert", "start_cones", out_just_file)
    @show B
    open(out_file_name, "w") do out
        open(in_file_name, "r") do inp
            run(pipeline(`gfan_tropicalstartingcone`, stdin=inp, stdout=out))
        end
    end
end
```

**Warning** the precomputed start cone files contain lines at the end that record the symmetries of the ideal. These symmetries were determined independently, and are not produced by the above code. The function `gfan_tropicaltraverse` automatically verifies that the listed symmetries are indeed symmetries of the ideal. These symmetries are essential for large examples, and we incldue them for most examples.  

Now we are ready to compute the tropicalizations of all Schubert varieties $X_B \subseteq \operatorname{Gr}(2,7)$ using `gfan_tropicaltraverse`. The following block of code does this, but without the symmetry feature. 

**Warning** in order for this to run, create the directory `precomputed_data/schubert/tropicalizations`. 

```
r = 2; n = 7; 
for B in sort!(subsets(n,r))
    if last(B) != n
        continue
    end
    Bs = join([string(a) for a in B])
    S = schubert_variety(r, n, B, true)
    if iszero(S)
        continue
    end
    in_just_file = "schubert_r$(r)_n$(n)_B$(Bs).dat"
    in_file_name = joinpath("precomputed_data/schubert", "start_cones", in_just_file)
    out_just_file = "schubert_r$(r)_n$(n)_B$(Bs).trop"
    out_file_name = joinpath("precomputed_data/schubert", "tropicalizations", out_just_file)
    
    open(out_file_name, "w") do out
        open(in_file_name, "r") do inp
            run(pipeline(`gfan_tropicaltraverse`, stdin=inp, stdout=out))
        end
    end
end
```

In [ ]:
r = 2; n = 7; 
for B in sort!(subsets(n,r))
    if last(B) != n
        continue
    end
    Bs = join([string(a) for a in B])
    S = schubert_variety(r, n, B, true)
    if iszero(S)
        continue
    end
    in_just_file = "schubert_r$(r)_n$(n)_B$(Bs).dat"
    in_file_name = joinpath("precomputed_data/schubert", "start_cones", in_just_file)
    out_just_file = "schubert_r$(r)_n$(n)_B$(Bs).trop"
    out_file_name = joinpath("precomputed_data/schubert", "tropicalizations", out_just_file)
    
    open(out_file_name, "w") do out
        open(in_file_name, "r") do inp
            run(pipeline(`gfan_tropicaltraverse`, stdin=inp, stdout=out))
        end
    end
end

We separately computed the tropicalizations using symmetry. This data may be found in the folder `precomputed_data/schubert/tropicalizations_nocones`. This is what we use to compute the fan $\Omega_t(X_B)$ for all Schubert varieties $X_B \subseteq \operatorname{Gr}(2,7)$, as we have done previously for $\Omega_t(X_{2,4,7})$.

In [ ]:
r = 2; n = 7; 
Brn = [[3,7],[4,7],[5,7],[6,7]]
for B in Brn
    IS = schubert_variety(r, n, B, true)
    if iszero(IS)
        continue
    end
    point_conf = schubert_lineality_space(r, n, B)
    Bs = join([string(a) for a in B])
    just_file = "schubert_r$(r)_n$(n)_B$(Bs).trop"
    file_name = joinpath("precomputed_data/schubert", "tropicalizations_nocones", just_file)
    rs, cs = parse_tropical_file(file_name, true)
    
    O = Omega_fan_t(IS, point_conf, [rs, cs])
    just_file = "Omega_tropical_no_saturation_cones_r$(r)_n$(n)_B$(Bs).mrdi"
    file_name = joinpath("precomputed_data/schubert", "omega_in_tropical_fan", just_file)
    #save(file_name, O[2]);
end


One technical detail that the above calculations of $\Omega_t(X_B)$ require is that each Gröbner cone of $\mathrm{Trop}(I(X_B))$ is contained in a secondary cone of $\mathcal{A}(I(X_B))$. The following function runs this test for a single $X_B$.

In [ ]:
function test_grob_in_sec(r, n, B)
    Bs = join([string(a) for a in B])
    S = schubert_variety(r, n, B, true)
    point_conf = point_configuration(S)
    L = transpose(schubert_lineality_space(r,n,B))
    just_file = "schubert_r$(r)_n$(n)_B$(Bs).trop"
    file_name = joinpath("precomputed_data/schubert", "tropicalizations_nocones", just_file)
    rs, cs = parse_tropical_file(file_name, true)
    ws = [-Vector{Int64}(pm.common.primitive(w)) for w in ray_cone_to_reps(rs, cs)]
    tests = []
    for i in 1:length(ws)
        if iszero(ws[i])
            push!(tests, true)
            continue
        end
        subd = subdivision_of_points(point_conf, ws[i])
        sec_cone = secondary_cone(subd)
        grob_cone = positive_hull(-rs[cs[i,:]], L)
        push!(tests, is_subset(grob_cone, sec_cone))
    end
    return all(tests)
end

We list all values of $B$ such that $I(X_B)$ is not toric. Note that $X_B$ does not depend on $n$.

In [ ]:
Bs_n6 = [[1,4,6],[1,5,6],[2,4,6],[2,5,6],[3,4,6],[3,5,6],[4,5,6]];
Bs_n7 = [[1,4,7],[1,5,7],[1,6,7],[2,4,7],[2,5,7],[2,6,7],[3,4,7],[3,5,7],[3,6,7],[4,5,7],[4,6,7],[5,6,7]];

We run the test for each $X_B$. 

In [ ]:
for B in Bs_n6
    println(B, " ", test_grob_in_sec(3, 6, B))
end

In [ ]:
for B in Bs_n7
    println(B, " ", test_grob_in_sec(3, 7, B))
end